In [ ]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatAnthropic

llm = OpenAI()
chat = ChatAnthropic()

llm.predict("How many planets are there")
chat.predict("How many planets are there")